## 1. 데이터 확인

### 필수 라이브러리

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore') # 경고 메세지 무시

In [3]:
df_train_origin = pd.read_csv("./data/train.csv") # 학습용 데이터
df_test_origin = pd.read_csv("./data/submission.csv") # 테스트 데이터(제출파일의 데이터)

In [4]:
df_train_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

## 2. 데이터 전처리

### 각 변수별 확인

In [5]:
df_train_process = pd.read_csv('./data/Ch2/df_train.csv')
df_test_process = pd.read_csv('./data/Ch2/df_test.csv')

In [6]:
df_train_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         59299 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            59299 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  59299 non-null  float64
 8   id_strategic_ver         59299 non-null  float64
 9   it_strategic_ver         59299 non-null  float64
 10  idit_strategic_ver       59299 non-null  float64
 11  customer_job             59299 non-null  object 
 12  lead_desc_length         59299 non-null  float64
 13  inquiry_type             59299 non-null  object 
 14  product_category      

## 3. 피처엔지니어링

### 레이블 인코딩

In [7]:
df_train_encoded = pd.read_csv('./data/Ch3/df_train_encoded.csv')
df_test_encoded = pd.read_csv('./data/Ch3/df_test_encoded.csv')

In [8]:
df_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   business_unit            59299 non-null  float64
 2   com_reg_ver_win_rate     59299 non-null  float64
 3   customer_idx             59299 non-null  float64
 4   customer_type            59299 non-null  float64
 5   enterprise               59299 non-null  float64
 6   historical_existing_cnt  59299 non-null  float64
 7   id_strategic_ver         59299 non-null  float64
 8   it_strategic_ver         59299 non-null  float64
 9   idit_strategic_ver       59299 non-null  float64
 10  customer_job             59299 non-null  float64
 11  lead_desc_length         59299 non-null  float64
 12  inquiry_type             59299 non-null  float64
 13  product_category         59299 non-null  float64
 14  product_subcategory   

## 4. 모델링

### 데이터 분할

학습, 검증 데이터 분리

In [9]:
# !pip install imblearn

In [10]:
from sklearn.model_selection import train_test_split

df_train_encoded.loc[df_train_encoded['is_converted'] == True, 'is_converted'] = 1
df_train_encoded.loc[df_train_encoded['is_converted'] == False, 'is_converted'] = 0

df_train_encoded['is_converted'] = df_train_encoded['is_converted'].astype(float)

X = df_train_encoded[df_train_encoded.columns.drop('is_converted')]
Y = df_train_encoded['is_converted']

# X와 Y로 나누기
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=0, shuffle=True)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted"], axis=1)

### 오버샘플링 & 언더샘플링

In [11]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# SMOTE와 RandomUnderSampler를 파이프라인으로 결합
resample = Pipeline([('SMOTE', SMOTE(random_state=0)), 
                     ('RandomUnderSampler', RandomUnderSampler(random_state=0))])

# 데이터에 오버샘플링과 언더샘플링 적용
x_train, y_train = resample.fit_resample(x_train, y_train)


### 모델 성능 보기

In [12]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])
    weighted_F1 = f1_score(y_test, y_pred, average='weighted')

    metrics = pd.DataFrame({
        '정확도': [accuracy],
        '정밀도': [precision],
        '재현율': [recall],
        'F1 Score': [F1],
        'Weighted F1': [weighted_F1]
    })

    confusion_df = pd.DataFrame(confusion, index=['True', 'False'], columns=['True', 'False'])

    print("\n오차행렬:")
    display(confusion_df)
    print("평가 지표:")
    display(metrics)


### 모델 라이브러리

단일모델 기준으로 사용할수 있는 모델들의 라이브러리를 불러오는 코드에 작성

In [13]:
# 모델 라이브러리
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

# 보팅
from sklearn.ensemble import VotingClassifier

### 최적 하이퍼 파라미터 찾기

optuna를 통한 최적의 파라미터 찾기

In [14]:
## 파라미터 찾는 과정.. ##

다시한번 optuna를 통한 최적의 파라미터 찾는과정

### 모델 정의

#### 단일 모델

In [15]:
# RandomForest
rf_model = RandomForestClassifier(
    n_estimators=1935
    , max_depth=32
    , min_samples_split=2
    , min_samples_leaf=1   
    , bootstrap=True
    , criterion='entropy'
    , class_weight='balanced'
    , random_state=0
)

# LightGBM_dart
lgbm_dart_model = LGBMClassifier(
    n_estimators=1029
    , num_leaves=167
    , max_depth=30
    , learning_rate=0.05767571715999541
    , min_child_samples=25
    , verbose=-1
    , boosting='dart'  # dart 사용
    , random_state=0
)

# XGBoost 
xgb_model = XGBClassifier(
    n_estimators=414
    , learning_rate=0.20046808426888615
    , max_depth=11
    , alpha=0.004365542651458743
    , gamma=0.00025712949731685885
    , reg_alpha=0.17168922089033928
    , reg_lambda=0.03881395024846057
    , colsample_bytree=0.32031741412326675
    , subsample=0.6269215430592496
    , objective='binary:logistic'  # 이진 분류
    , tree_method="exact"        
    , random_state=0
)

# ExtraTrees
et_model = ExtraTreesClassifier(
    n_estimators=100
    , max_depth=48
    , min_samples_split=3
	, min_samples_leaf=1
	, criterion='gini'
	, random_state=0
)

### 모델 학습

In [16]:
### 보팅 분류기 생성 ###
model = VotingClassifier(
    estimators=[
        ('lgb_dart',lgbm_dart_model),
        ('xgb', xgb_model),
        ('rf', rf_model),
        ('et', et_model)
    ],
    voting='soft'  # 'hard'는 다수결 투표, 'soft'는 확률 평균
)

In [17]:
model.fit(x_train, y_train)

VotingClassifier(estimators=[('lgb_dart',
                              LGBMClassifier(boosting='dart',
                                             learning_rate=0.05767571715999541,
                                             max_depth=30, min_child_samples=25,
                                             n_estimators=1029, num_leaves=167,
                                             random_state=0, verbose=-1)),
                             ('xgb',
                              XGBClassifier(alpha=0.004365542651458743,
                                            base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_by...
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=414, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                             ('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     criterion='entropy',
                                                     max_depth=32,
                                                     n_estimators=1935,
                                                     random_state=0)),
                             ('et',
                              ExtraTreesClassifier(max_depth=48,
                                                   min_samples_split=3,
                                                   random_state=0))],
                 voting='soft')

In [18]:
# 확률 예측
soft_voting_probs = model.predict_proba(x_val)[:, 1]

# 스레시홀드 0.12 적용
soft_voting_preds = [1 if prob > 0.12 else 0 for prob in soft_voting_probs]

# 평가
get_clf_eval(y_val, soft_voting_preds)


오차행렬:


,True,False
True,949,45
False,523,10343


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.952108,0.644701,0.954728,0.769667,0.956211


In [19]:
# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

# 변경된 스레시홀드 0.12을 적용하여 테스트 데이터에 대한 예측
soft_voting_probs = model.predict_proba(x_test)[:, 1]
soft_voting_preds = [1 if prob > 0.12 else 0 for prob in soft_voting_probs]

# 테스트 데이터에서 True로 예측된 개수 출력
sum(soft_voting_preds)

1986

### 제출 파일 작성

In [83]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["is_converted"] = soft_voting_preds

# 제출 파일 저장
df_sub.to_csv("submission_0.76077.csv", index=False)

.